In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
class LiverFibrosisDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                if img_path.endswith(".jpg"):
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "/kaggle/input/liver-histopathology-fibrosis-ultrasound-images/Dataset/Dataset"

In [5]:
full_dataset = LiverFibrosisDataset(dataset_path, transform=transform)

In [6]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

In [7]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet121(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 156MB/s] 


In [9]:
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 5)
model = model.to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [11]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, save_path="densenet_fibrosis.pth"):
    best_val_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = 100 * correct / total
        val_acc, val_loss = evaluate_model(model, val_loader, criterion)
        
        print(f"Epoch {epoch+1}/{epochs}: Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print(f"Model saved with Validation Accuracy: {val_acc:.2f}%")


In [12]:
def evaluate_model(model, data_loader, criterion):
    model.eval()
    correct, total = 0, 0
    running_loss = 0.0
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = 100 * correct / total
    return acc, running_loss / len(data_loader)

In [13]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, save_path="densenet_fibrosis.pth")

Epoch 1/10: Train Loss: 0.6386, Train Acc: 73.23%, Val Loss: 1.5356, Val Acc: 41.29%
Model saved with Validation Accuracy: 41.29%
Epoch 2/10: Train Loss: 0.4750, Train Acc: 80.57%, Val Loss: 0.6359, Val Acc: 78.99%
Model saved with Validation Accuracy: 78.99%
Epoch 3/10: Train Loss: 0.4013, Train Acc: 84.25%, Val Loss: 0.6854, Val Acc: 76.02%
Epoch 4/10: Train Loss: 0.3698, Train Acc: 85.36%, Val Loss: 0.7991, Val Acc: 74.91%
Epoch 5/10: Train Loss: 0.3491, Train Acc: 86.58%, Val Loss: 0.4898, Val Acc: 84.30%
Model saved with Validation Accuracy: 84.30%
Epoch 6/10: Train Loss: 0.2536, Train Acc: 91.71%, Val Loss: 0.5851, Val Acc: 80.47%
Epoch 7/10: Train Loss: 0.3091, Train Acc: 90.07%, Val Loss: 0.3488, Val Acc: 86.77%
Model saved with Validation Accuracy: 86.77%
Epoch 8/10: Train Loss: 0.1878, Train Acc: 93.78%, Val Loss: 0.7270, Val Acc: 79.73%
Epoch 9/10: Train Loss: 0.2475, Train Acc: 91.69%, Val Loss: 0.3998, Val Acc: 85.17%
Epoch 10/10: Train Loss: 0.1672, Train Acc: 96.08%, Val

In [14]:
model.load_state_dict(torch.load("densenet_fibrosis.pth"))
model.to(device)
print("Best model loaded for testing.")

Best model loaded for testing.


<ipython-input-14-bfec91903225>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("densenet_fibrosis.pth"))


In [15]:
def test_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["F0", "F1", "F2", "F3", "F4"]))

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

In [16]:
test_model(model, test_loader)

Classification Report:
              precision    recall  f1-score   support

          F0       1.00      0.98      0.99       320
          F1       0.64      0.90      0.74       107
          F2       0.71      0.48      0.57        77
          F3       0.85      0.78      0.82        68
          F4       0.97      0.94      0.96       239

    accuracy                           0.90       811
   macro avg       0.84      0.82      0.82       811
weighted avg       0.90      0.90      0.89       811

Confusion Matrix:
[[315   5   0   0   0]
 [  0  96   6   2   3]
 [  0  33  37   7   0]
 [  0   7   5  53   3]
 [  0  10   4   0 225]]
